<a href="https://www.kaggle.com/code/shariq20220/cv-project-tripletloss?scriptVersionId=191351193" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%pip install wildlife-datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 679.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install wildlife-tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.8/427.8 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 8.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install timm

Note: you may need to restart the kernel to use updated packages.


In [4]:
from itertools import chain
import timm
import pandas as pd
import torchvision.transforms as T
from torch.optim import SGD

from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.train import ArcFaceLoss, BasicTrainer

import timm
import numpy as np
from wildlife_datasets.datasets import MacaqueFaces
from wildlife_tools.data import WildlifeDataset
import torchvision.transforms as T
from wildlife_datasets import datasets, splits
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

import timm
import itertools
from torch.optim import SGD
from wildlife_tools.train import ArcFaceLoss, BasicTrainer , TripletLoss

2024-08-06 03:39:23.937098: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-06 03:39:23.937265: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-06 03:39:24.061258: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


**IPanda50 Dataset**

In [5]:


# Download dataset (if not already downloaded)
datasets.IPanda50.get_data('../data/IPanda50')
# Load dataset metadata
metadata = datasets.IPanda50('../data/IPanda50')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = TripletLoss()

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()



DATASET IPanda50: DOWNLOADING STARTED.


Downloading...
From (original): https://drive.google.com/uc?id=1nkh-g6a8JvWy-XsMaZqrN2AXoPlaXuFg
From (redirected): https://drive.google.com/uc?id=1nkh-g6a8JvWy-XsMaZqrN2AXoPlaXuFg&confirm=t&uuid=91935a56-3c4b-4f8d-a677-f3fc46e4fcbb
To: /kaggle/data/IPanda50/iPanda50-images.zip
100%|██████████| 926M/926M [00:06<00:00, 148MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gVREtFWkNec4xwqOyKkpuIQIyWU_Y_Ob
To: /kaggle/data/IPanda50/iPanda50-split.zip
100%|██████████| 140k/140k [00:00<00:00, 41.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1jdACN98uOxedZDT-6X3rpbooLAAUEbNY
To: /kaggle/data/IPanda50/iPanda50-eyes-labels.zip
100%|██████████| 2.55M/2.55M [00:00<00:00, 196MB/s]


DATASET IPanda50: EXTRACTING STARTED.
DATASET IPanda50: FINISHED.



config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/204M [00:00<?, ?B/s]

...

Epoch 18: Average Loss = 0.19903765618801117


Epoch 18: 100%|█████████████████████████████████████████████████████| 54/54 [01:22<00:00,  1.53s/it]


Epoch 19: Average Loss = 0.1964729130268097


Epoch 19: 100%|█████████████████████████████████████████████████████| 54/54 [01:22<00:00,  1.53s/it]

Epoch 20: Average Loss = 0.20050834119319916


In [6]:
import torch

trainer.save("retrained_chks", file_name="tripletloss_IPanda50_retrained_checkpoint.pth")


In [7]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(backbone , device = 'cuda')

query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)


100%|███████████████████████████████████████████████████████████████| 53/53 [01:05<00:00,  1.24s/it]


In [8]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)


{'cosine': array([[-2.06143856e-02,  4.78334099e-01,  1.32662226e-02, ...,
         3.80642682e-01,  3.45688283e-01,  2.54310202e-04],
       [ 3.48474011e-02,  5.98875582e-01, -9.97744054e-02, ...,
         2.10135087e-01,  6.41717672e-01, -1.41400337e-01],
       [-5.76439574e-02,  7.77825117e-02,  9.52074230e-02, ...,
         4.88989204e-02,  2.31894881e-01, -1.06991403e-01],
       ...,
       [ 1.33126915e-01,  1.15358315e-01, -5.98059818e-02, ...,
         4.07572314e-02,  1.04062401e-01, -1.07376292e-01],
       [-7.23384470e-02,  2.52226368e-02,  7.67257884e-02, ...,
        -4.85982411e-02,  1.68619499e-01, -2.34705042e-02],
       [-3.54364440e-02,  4.61052731e-02,  6.25129879e-01, ...,
         1.83079273e-01,  5.30509278e-05,  1.87101886e-02]], dtype=float32)}


In [9]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)


Predictions for 100 test Images:-
 ['19_nannan' '37_xinger' '18_miaomiao' '33_wuyi' '47_yingying' '15_maosun'
 '09_fushun' '10_hexing' '36_xingda' '19_nannan' '08_fulai' '31_shurong'
 '44_yayi' '39_xinghui' '28_sa' '14_maodou' '14_maodou' '00_aibang'
 '21_nina' '35_xilan' '19_nannan' '33_wuyi' '47_yingying' '49_yuanrun'
 '36_xingda' '34_xiaoqiao' '16_maotao' '48_yongbang' '11_jiaoao'
 '49_yuanrun' '44_yayi' '28_sa' '35_xilan' '35_xilan' '02_baolan'
 '31_shurong' '31_shurong' '02_baolan' '23_qiubang' '26_qiyuan'
 '34_xiaoqiao' '35_xilan' '08_fulai' '00_aibang' '28_sa' '11_jiaoao'
 '46_yazhu' '44_yayi' '14_maodou' '30_shuqing' '36_xingda' '38_xingfan'
 '33_wuyi' '10_hexing' '10_hexing' '00_aibang' '49_yuanrun' '35_xilan'
 '04_chengdui' '10_hexing' '39_xinghui' '36_xingda' '10_hexing'
 '36_xingda' '16_maotao' '22_nini' '00_aibang' '39_xinghui' '10_hexing'
 '17_meibang' '00_aibang' '28_sa' '14_maodou' '15_maosun' '32_susu'
 '26_qiyuan' '27_rourou' '23_qiubang' '18_miaomiao' '48_yongbang'
 

/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [10]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on IPanda50 data: {:.2f}%".format(accuracy_P * 100))


Accuracy on IPanda50 data: 92.00%


In [11]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 0.9633333333333334
Recall: 0.92
F1 Score: 0.9208989898989899


**MacaqueFaces Dataset**

In [12]:
# Download dataset (if not already downloaded)
datasets.MacaqueFaces.get_data('../data/MacaqueFaces')
# Load dataset metadata
metadata = datasets.MacaqueFaces('../data/MacaqueFaces')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = TripletLoss()

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET MacaqueFaces: DOWNLOADING STARTED.


MacaqueFaces.zip: 12.0MB [00:00, 15.3MB/s]                            
MacaqueFaces_ImageInfo.csv: 410kB [00:00, 1.08MB/s]                            


DATASET MacaqueFaces: EXTRACTING STARTED.
DATASET MacaqueFaces: FINISHED.



Epoch 0:   0%|                                                               | 0/50 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0:  98%|████████████████████████████████████████████████████▉ | 49/50 [01:12<00:01,  1.48s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 50/50 [01:12<00:00,  1.46s/it]


Epoch 1: Average Loss = 0.20605501532554626


...

Epoch 18: Average Loss = 0.13126236200332642


Epoch 18: 100%|█████████████████████████████████████████████████████| 50/50 [01:09<00:00,  1.39s/it]


Epoch 19: Average Loss = 0.152557834982872


Epoch 19: 100%|█████████████████████████████████████████████████████| 50/50 [01:09<00:00,  1.39s/it]

Epoch 20: Average Loss = 0.13818107545375824


In [13]:

import torch

trainer.save("retrained_chks", file_name="tripletloss_MacaqueFaces_retrained_checkpoint.pth")

In [14]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(backbone , device = 'cuda')

query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

100%|███████████████████████████████████████████████████████████████| 49/49 [00:27<00:00,  1.81it/s]

{'cosine': array([[0.95115745, 0.9641235 , 0.95786726, ..., 0.0564175 , 0.06427768,
        0.10707347],
       [0.96187276, 0.9701442 , 0.96455514, ..., 0.0986455 , 0.1119099 ,
        0.15456885],
       [0.93769866, 0.9570849 , 0.95075125, ..., 0.09609564, 0.10794473,
        0.14246391],
       ...,
       [0.93491936, 0.97786546, 0.97431666, ..., 0.09593339, 0.098791  ,
        0.14425984],
       [0.91334474, 0.97317624, 0.9620143 , ..., 0.06551258, 0.06374791,
        0.11121601],
       [0.9444787 , 0.9582597 , 0.94499683, ..., 0.18544054, 0.19311081,
        0.23016736]], dtype=float32)}


In [15]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [16]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on MacaqueFaces data: {:.2f}%".format(accuracy_P * 100))

Accuracy on MacaqueFaces data: 100.00%


In [17]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**LionData Dateset**

In [18]:
# Download dataset (if not already downloaded)
datasets.LionData.get_data('../data/LionData')
# Load dataset metadata
metadata = datasets.LionData('../data/LionData')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = TripletLoss()

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET LionData: DOWNLOADING STARTED.


main.zip: 495MB [00:21, 23.2MB/s]


DATASET LionData: EXTRACTING STARTED.
DATASET LionData: FINISHED.



Epoch 0:   0%|                                                                | 0/6 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|████████████████████████████████████████████████████████| 6/6 [00:09<00:00,  1.42s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|████████████████████████████████████████████████████████| 6/6 [00:09<00:00,  1.52s/it]


Epoch 1: Average Loss = 0.34933146834373474


...

Epoch 18: Average Loss = 0.271456241607666


Epoch 18: 100%|███████████████████████████████████████████████████████| 6/6 [00:09<00:00,  1.54s/it]


Epoch 19: Average Loss = 0.2715429365634918


Epoch 19: 100%|███████████████████████████████████████████████████████| 6/6 [00:09<00:00,  1.54s/it]

Epoch 20: Average Loss = 0.25652801990509033


In [19]:
import torch

trainer.save("retrained_chks", file_name="tripletloss_LionData_retrained_checkpoint.pth")

In [20]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(backbone , device = 'cuda')

query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)


similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

100%|█████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.53it/s]

{'cosine': array([[0.58438665, 0.5151248 , 0.63837945, ..., 0.6976384 , 0.70911604,
        0.5498905 ],
       [0.78872794, 0.45712468, 0.7232994 , ..., 0.48549446, 0.80339277,
        0.5131808 ],
       [0.4228299 , 0.43705624, 0.5581653 , ..., 0.5786077 , 0.6387228 ,
        0.7014551 ],
       ...,
       [0.4319855 , 0.68674433, 0.5633152 , ..., 0.74344164, 0.5669186 ,
        0.5497502 ],
       [0.39819872, 0.46327004, 0.51167727, ..., 0.61548626, 0.6160317 ,
        0.83589125],
       [0.4491108 , 0.5372858 , 0.5421016 , ..., 0.7385289 , 0.6288502 ,
        0.73790455]], dtype=float32)}


In [21]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['Sebastian' 'Saba' 'Sangiki' 'Tigisi' 'Saimutie' 'Moswen' 'Enderoni'
 'Simaloi' 'Spring' 'Chaimu' 'Maskio' 'Manzy' 'Kaka' 'Saba' 'Saitoti'
 'Joy' 'Spring' 'Etana' 'Enderoni' 'Kinna' 'Senteu' 'Enkume' 'Joy' 'Joy'
 'Naibor' 'Romeo' 'Doto' 'Selenkay' 'Sadala' 'Manzy' 'Moswen' 'Lucinda'
 'Mooza' 'Naape' 'Napi' 'Kibibi' 'Nasieku' 'Kioni' 'Engiyaa' 'Nesapuka'
 'Samir' 'Jamal' 'Leia' 'Sebastian' 'Yieyio' 'Saimutie' 'Nguro' 'Moswen'
 'Kibibi' 'Naini' 'Enderoni' 'Lucinda' 'Kinna' 'Nuru' 'Sikio-Kali'
 'Mashavu' 'Tigisi' 'Nguro' 'Lilly' 'Manzy' 'Opi' 'Sikio-Kali' 'Ngare'
 'Doto' 'Summer' 'Nasieku' 'Nakato' 'Kioni' 'Chaimu' 'Amber' 'Shemsa'
 'Engiyaa' 'Mickey' 'White-Eye' 'Naimina' 'Tigisi' 'Summer' 'Naibor'
 'Sangiki' 'Nariku-Inkgera' 'Cleopatra' 'Naape' 'Naisiae' 'Moswen'
 'Lucinda' 'Naini' 'Rescue' 'Kinna' 'Empurra' 'Enadalut' 'Blackie'
 'Simaloi' 'Lucinda' 'Yieyio' 'Shambe' 'Enderoni' 'Maskio' 'Enkume' 'Nuru'
 'Ngare']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [22]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on LionData data: {:.2f}%".format(accuracy_P * 100))

Accuracy on LionData data: 14.00%


In [23]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 0.49583333333333335
Recall: 0.14
F1 Score: 0.12890476190476188


**NyalaDataSet**

In [24]:
# Download dataset (if not already downloaded)
datasets.NyalaData.get_data('../data/NyalaData')
# Load dataset metadata
metadata = datasets.NyalaData('../data/NyalaData')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = TripletLoss()

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET NyalaData: DOWNLOADING STARTED.


main.zip: 495MB [00:19, 25.1MB/s]


DATASET NyalaData: EXTRACTING STARTED.
DATASET NyalaData: FINISHED.



Epoch 0:   0%|                                                               | 0/16 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0:  94%|██████████████████████████████████████████████████▋   | 15/16 [00:50<00:03,  3.33s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 16/16 [00:50<00:00,  3.16s/it]


Epoch 1: Average Loss = 0.27757370471954346


...

Epoch 18: Average Loss = 0.2240530252456665


Epoch 18: 100%|█████████████████████████████████████████████████████| 16/16 [00:50<00:00,  3.16s/it]


Epoch 19: Average Loss = 0.22677916288375854


Epoch 19: 100%|█████████████████████████████████████████████████████| 16/16 [00:49<00:00,  3.09s/it]

Epoch 20: Average Loss = 0.2326413094997406


In [25]:
import torch

trainer.save("retrained_chks", file_name="tripletloss_LionData_retrained_checkpoint.pth")

In [26]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(backbone , device = 'cuda')

query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)


similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

100%|███████████████████████████████████████████████████████████████| 15/15 [00:46<00:00,  3.08s/it]

{'cosine': array([[0.71855605, 0.25089562, 0.17059799, ..., 0.48864168, 0.16155   ,
        0.77021146],
       [0.47162133, 0.77640796, 0.6154964 , ..., 0.3920012 , 0.53229874,
        0.33269235],
       [0.72394395, 0.4533545 , 0.4138953 , ..., 0.92586684, 0.16884752,
        0.5211943 ],
       ...,
       [0.333907  , 0.79697764, 0.65271693, ..., 0.25404117, 0.6962526 ,
        0.2702029 ],
       [0.72142595, 0.23822626, 0.22308147, ..., 0.70529175, 0.1139962 ,
        0.8042281 ],
       [0.13170072, 0.540874  , 0.76094866, ..., 0.11455392, 0.84263206,
        0.13025302]], dtype=float32)}


In [27]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['62' '10' '29' '149' '144' '179' '147' '28' '48' '71' '114' '44' '33'
 '144' '61' '25' '131' '272' '61' '68' '116' '127' '104' '268' '260' '30'
 '111' '11' '188' '46' '88' '114' '111' '131' '88' '209' '247' '43' '152'
 '7' '272' '234' '48' '46' '8' '122' '114' '62' '260' '144' '115' '151'
 '27' '108' '4' '252' '104' '85' '32' '140' '46' '48' '63' '85' '97' '181'
 '65' '242' '11' '89' '113' '145' '60' '17' '127' '166' '240' '54' '27'
 '36' '20' '149' '23' '85' '196' '20' '268' '62' '165' '106' '100' '174'
 '149' '217' '33' '225' '50' '85' '40' '8']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [28]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on NyalaData data: {:.2f}%".format(accuracy_P * 100))

Accuracy on NyalaData data: 20.00%


In [29]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 0.6083333333333333
Recall: 0.2
F1 Score: 0.21752380952380954


**StripeSpotter Dataset**

In [30]:
# Download dataset (if not already downloaded)
datasets.StripeSpotter.get_data('../data/StripeSpotter')
# Load dataset metadata
metadata = datasets.StripeSpotter('../data/StripeSpotter')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = TripletLoss()

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET StripeSpotter: DOWNLOADING STARTED.


data-20110718.zip: 71.7MB [00:00, 136MB/s]                            
data-20110718.z02: 100%|██████████| 78.6M/78.6M [00:00<00:00, 124MB/s]
data-20110718.z01: 100%|██████████| 78.6M/78.6M [00:00<00:00, 109MB/s]


DATASET StripeSpotter: EXTRACTING STARTED.
 copying: data/
 copying: data/.DS_Store
 copying: data/images/
 copying: data/images/img-0000001.jpg
 copying: data/images/img-0000002.jpg
 copying: data/images/img-0000003.jpg
 copying: data/images/img-0000004.jpg
 copying: data/images/img-0000005.jpg
 copying: data/images/img-0000006.jpg
 copying: data/images/img-0000007.jpg
 copying: data/images/img-0000008.jpg
 copying: data/images/img-0000009.jpg
 copying: data/images/img-0000010.jpg
 copying: data/images/img-0000011.jpg
 copying: data/images/img-0000012.jpg
 copying: data/images/img-0000013.jpg
 copying: data/images/img-0000014.jpg
 copying: data/images/img-0000015.jpg
 copying: data/images/img-0000016.jpg
 copying: data/images/img-0000017.jpg
 copying: data/images/img-0000018.jpg
 copying: data/images/img-0000019.jpg
 copying: data/images/img-0000020.jpg
 copying: data/images/img-0000021.jpg
 copying: data/images/img-0000022.jpg
 copying: data/images/img-0000023.jpg
 copying: data/imag

Epoch 0:   0%|                                                                | 0/7 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0:  86%|████████████████████████████████████████████████        | 6/7 [00:23<00:03,  3.66s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|████████████████████████████████████████████████████████| 7/7 [00:23<00:00,  3.41s/it]


Epoch 1: Average Loss = 0.26624637842178345


...

Epoch 18: Average Loss = 0.17158709466457367


Epoch 18: 100%|███████████████████████████████████████████████████████| 7/7 [00:23<00:00,  3.41s/it]


Epoch 19: Average Loss = 0.17276084423065186


Epoch 19: 100%|███████████████████████████████████████████████████████| 7/7 [00:23<00:00,  3.39s/it]

Epoch 20: Average Loss = 0.16557720303535461


In [31]:
import torch

trainer.save("retrained_chks", file_name="tripletloss_LionData_retrained_checkpoint.pth")

In [32]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(backbone , device = 'cuda')

query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)


similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

100%|█████████████████████████████████████████████████████████████████| 6/6 [00:20<00:00,  3.43s/it]

{'cosine': array([[-0.06076185,  0.2354279 ,  0.1241855 , ...,  0.03633047,
         0.24368721,  0.40893114],
       [-0.0041027 ,  0.42128778,  0.4651789 , ...,  0.5937002 ,
         0.18138684,  0.11973396],
       [ 0.5033885 ,  0.11032735,  0.01200732, ...,  0.02336629,
         0.15754193,  0.01167232],
       ...,
       [ 0.34611425, -0.053916  , -0.14451835, ..., -0.0472455 ,
         0.11016403, -0.03051265],
       [ 0.36755234, -0.07642552, -0.0417766 , ...,  0.03269858,
         0.16064902, -0.11850601],
       [-0.02964952,  0.07616474, -0.09410825, ...,  0.07067783,
         0.17747642,  0.00386247]], dtype=float32)}


In [33]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['09_200' 'S09_161' '08_085' '08_100' '09_200' 'OP05_625' '09_212'
 '02_149' '01_579' '08_034' '01_679' '08_078' '01_460' '10_002' '01_136'
 '09_212' '09_200' 'S09_111' '02_149' 'S09_069' '08_100' '01_363' '01_363'
 '08_078' '08_034' '01_805' '09_355' '08_085' 'OP05_546' '09_212' '08_100'
 'S09_088' '09_182' 'S09_215' '02_162' '01_230' '08_085' '01_661'
 '02_1026' '09_185' '01_136' '01_661' '01_661' '02_161' '01_165' '02_161'
 'S09_174' 'S09_067' '08_100' 'OP04_225' '02_149' 'S09_067' '09_200'
 '04_008' '01_661' '09_182' '01_805' '02_161' '10_002' '01_036' '02_149'
 '01_165' '09_212' 'S09_211' 'S09_215' '03_045' '01_579' 'OP04_225'
 '08_100' '08_100' '04_008' '09_212' 'OP05_625' 'OP05_625' '02_149'
 '01_230' '09_200' 'OP05_625' '01_661' 'OP04_225' '01_661' '09_212'
 'S09_174' '01_679' '01_661' 'S09_067' 'S09_111' 'S09_112' '09_212'
 '09_200' '02_161' '09_185' '04_008' '08_100' '01_460' '01_679' 'S09_111'
 '01_679' '01_036' 'S09_067']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [34]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on StripeSpotter data: {:.2f}%".format(accuracy_P * 100))

Accuracy on StripeSpotter data: 98.00%


In [35]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 0.9866666666666666
Recall: 0.98
F1 Score: 0.9766666666666667


**CZoo Dataset**

In [36]:
# Download dataset (if not already downloaded)
datasets.CZoo.get_data('../data/CZoo')
# Load dataset metadata
metadata = datasets.CZoo('../data/CZoo')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = TripletLoss()

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET CZoo: DOWNLOADING STARTED.


master.zip: 634MB [00:19, 32.0MB/s]


DATASET CZoo: EXTRACTING STARTED.
DATASET CZoo: FINISHED.



Epoch 0:   0%|                                                               | 0/17 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0:  94%|██████████████████████████████████████████████████▊   | 16/17 [00:25<00:01,  1.55s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 17/17 [00:26<00:00,  1.55s/it]


Epoch 1: Average Loss = 0.15043650567531586


...

Epoch 18: Average Loss = 0.12739834189414978


Epoch 18: 100%|█████████████████████████████████████████████████████| 17/17 [00:25<00:00,  1.49s/it]


Epoch 19: Average Loss = 0.11058130115270615


Epoch 19: 100%|█████████████████████████████████████████████████████| 17/17 [00:25<00:00,  1.47s/it]

Epoch 20: Average Loss = 0.1075560599565506


In [37]:
import torch

trainer.save("retrained_chks", file_name="tripletloss_LionData_retrained_checkpoint.pth")

In [38]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(backbone , device = 'cuda')

query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)


similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

100%|███████████████████████████████████████████████████████████████| 16/16 [00:15<00:00,  1.03it/s]

{'cosine': array([[ 0.87354   ,  0.87993264,  0.8596037 , ..., -0.02853326,
        -0.03062224,  0.0309791 ],
       [ 0.8830426 ,  0.89095914,  0.85238206, ..., -0.02522288,
        -0.04309715, -0.02871453],
       [ 0.8799157 ,  0.8908291 ,  0.8558865 , ..., -0.06945187,
        -0.03231023,  0.02483982],
       ...,
       [ 0.9066069 ,  0.93468696,  0.89549494, ..., -0.07639489,
        -0.00395182, -0.02399763],
       [ 0.9109483 ,  0.9139593 ,  0.8913132 , ..., -0.05694149,
        -0.01770366, -0.01955829],
       [ 0.8890263 ,  0.9301636 ,  0.9091662 , ..., -0.08943634,
         0.02013947, -0.01138124]], dtype=float32)}


In [39]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['Alex' 'Alex' 'Alex' 'Jahaga' 'Jahaga' 'Alex' 'Alex' 'Jahaga' 'Alex'
 'Jahaga' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex'
 'Alex' 'Alex' 'Alex' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Gertrudia' 'Gertrudia' 'Gertrudia' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Annett' 'Annett' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Al

/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [40]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on CZoo data: {:.2f}%".format(accuracy_P * 100))

Accuracy on CZoo data: 100.00%


In [41]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**CowDataset**

In [42]:
# Download dataset (if not already downloaded)
datasets.CowDataset.get_data('../data/CowDataset')
# Load dataset metadata
metadata = datasets.CowDataset('../data/CowDataset')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = TripletLoss()

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET CowDataset: DOWNLOADING STARTED.


31210192: 4.45GB [02:06, 35.2MB/s]                            


DATASET CowDataset: EXTRACTING STARTED.
DATASET CowDataset: FINISHED.



Epoch 0:   0%|                                                               | 0/12 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 12/12 [03:45<00:00, 17.26s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 12/12 [03:45<00:00, 18.82s/it]


Epoch 1: Average Loss = 0.22858136892318726


...

Epoch 18: Average Loss = 0.11212163418531418


Epoch 18: 100%|█████████████████████████████████████████████████████| 12/12 [03:15<00:00, 16.26s/it]


Epoch 19: Average Loss = 0.10893279314041138


Epoch 19: 100%|█████████████████████████████████████████████████████| 12/12 [03:52<00:00, 19.36s/it]

Epoch 20: Average Loss = 0.1156926229596138


In [43]:
import torch

trainer.save("retrained_chks", file_name="tripletloss_LionData_retrained_checkpoint.pth")

In [44]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(backbone , device = 'cuda')

query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)


similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

100%|███████████████████████████████████████████████████████████████| 11/11 [03:01<00:00, 16.51s/it]

{'cosine': array([[-0.00787083,  0.0310967 ,  0.14591771, ..., -0.01681634,
         0.09756517,  0.09112016],
       [-0.07344489, -0.05329438,  0.06180625, ..., -0.06789516,
         0.1392704 ,  0.16481152],
       [ 0.05363748,  0.09170404,  0.07950462, ..., -0.00178711,
         0.02702484,  0.0183273 ],
       ...,
       [ 0.00969155,  0.01916153,  0.14822248, ..., -0.02853924,
         0.13578391,  0.09585967],
       [-0.10385527,  0.07502653,  0.07272013, ...,  0.00844184,
        -0.12051392, -0.13555706],
       [-0.02091158,  0.08030448,  0.03012463, ...,  0.20119372,
         0.65597034,  0.7598175 ]], dtype=float32)}


In [45]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['2' '9' '2' '13' '1' '6' '4' '7' '1' '4' '10' '9' '13' '3' '6' '10' '6'
 '4' '2' '13' '6' '2' '2' '9' '8' '8' '4' '7' '2' '12' '6' '6' '2' '4' '5'
 '2' '7' '9' '4' '5' '9' '11' '1' '8' '8' '1' '9' '4' '3' '7' '12' '1' '2'
 '5' '5' '9' '4' '1' '10' '3' '11' '12' '5' '3' '6' '5' '5' '2' '7' '11'
 '8' '5' '3' '9' '8' '7' '2' '1' '11' '1' '6' '3' '12' '6' '3' '8' '10'
 '11' '7' '6' '1' '8' '6' '3' '11' '10' '10' '2' '3' '13']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [46]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on CowDataset data: {:.2f}%".format(accuracy_P * 100))

Accuracy on CowDataset data: 100.00%


In [47]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 1.0
Recall: 1.0
F1 Score: 1.0


# **ON New Unseen Datasets**

**DogFaceNet Dataset**

In [48]:
# Download dataset (if not already downloaded)
datasets.DogFaceNet.get_data('../data/DogFaceNet')

# Load dataset metadata
metadata = datasets.DogFaceNet('../data/DogFaceNet')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = TripletLoss()

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET DogFaceNet: DOWNLOADING STARTED.


DogFaceNet_Dataset_224_1.zip: 75.6MB [00:00, 87.8MB/s]                           


DATASET DogFaceNet: EXTRACTING STARTED.
DATASET DogFaceNet: FINISHED.



Epoch 0:   0%|                                                               | 0/66 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0:  98%|█████████████████████████████████████████████████████▏| 65/66 [01:37<00:01,  1.51s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 66/66 [01:38<00:00,  1.49s/it]


Epoch 1: Average Loss = 0.16777630150318146


...

Epoch 18: Average Loss = 0.13342444598674774


Epoch 18: 100%|█████████████████████████████████████████████████████| 66/66 [01:33<00:00,  1.42s/it]


Epoch 19: Average Loss = 0.13554422557353973


Epoch 19: 100%|█████████████████████████████████████████████████████| 66/66 [01:33<00:00,  1.42s/it]

Epoch 20: Average Loss = 0.146966814994812


In [49]:
import torch

trainer.save("retrained_chks", file_name="tripletloss_DogFaceNet_retrained_checkpoint.pth")

In [50]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(backbone , device = 'cuda')

query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)


100%|███████████████████████████████████████████████████████████████| 65/65 [00:36<00:00,  1.80it/s]


In [51]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[-0.019843  ,  0.36029592,  0.00885892, ...,  0.35406327,
         0.40041047,  0.02531504],
       [-0.10484669,  0.01207395, -0.08835223, ...,  0.17490536,
         0.08758179, -0.0870728 ],
       [-0.0132006 , -0.17365316,  0.07511273, ...,  0.04451306,
         0.1492618 ,  0.21425228],
       ...,
       [ 0.27489462,  0.01977136,  0.31241396, ...,  0.16428486,
        -0.08841489,  0.40615654],
       [-0.01623731,  0.39576897, -0.01582699, ...,  0.19240376,
         0.30423856,  0.09543777],
       [ 0.03524115,  0.34433937,  0.05614508, ...,  0.21675274,
         0.32163084,  0.10202848]], dtype=float32)}


In [52]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['322' '182' '919' '1122' '428' '378' '1063' '272' '470' '405' '764' '453'
 '635' '1339' '432' '673' '1075' '274' '296' '234' '1272' '991' '485'
 '425' '280' '407' '258' '1248' '446' '896' '892' '448' '240' '251' '448'
 '470' '321' '189' '345' '763' '622' '1324' '158' '127' '1414' '1254'
 '1139' '208' '608' '424' '293' '415' '825' '147' '373' '192' '333' '333'
 '1314' '957' '329' '89' '189' '100' '122' '631' '221' '646' '26' '1369'
 '878' '673' '174' '447' '1169' '458' '473' '149' '373' '622' '540' '919'
 '513' '913' '782' '1357' '361' '266' '1324' '423' '183' '200' '299' '191'
 '397' '187' '270' '563' '1020' '20']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [53]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on DogFaceNet data: {:.2f}%".format(accuracy_P * 100))

Accuracy on DogFaceNet data: 80.00%


In [54]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 0.99
Recall: 0.8
F1 Score: 0.7933333333333333
